In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c digit-recognizer

In [ ]:
!unzip /content/digit-recognizer.zip

In [ ]:
train=pd.read_csv('train.csv')

In [ ]:
x=train.iloc[0:5000,1:]
y=train.iloc[0:5000,:1]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,stratify=y,random_state=42)

#Represent the digit as image

In [ ]:
import matplotlib.pyplot as plt, matplotlib.image as mpimg
i=1
img=x_train.iloc[i].to_numpy()
img=img.reshape((28,28))
plt.imshow(img,cmap='gray')

In [ ]:
x_train=x_train.values.reshape((-1,28,28,1))
x_test=x_test.values.reshape(-1,28,28,1)


In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)


Model cnn

In [ ]:
model_cnn=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',input_shape=(28,28,1),padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu',padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    #    tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',padding='same'),
    # tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    #       tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu',padding='same'),
    # tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')

])

In [ ]:
model_cnn.compile(loss='CategoricalCrossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=.001,beta_1=.9,beta_2=.99),
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])
early_stop=tf.keras.callbacks.EarlyStopping(patience=5)
lr=tf.keras.callbacks.ReduceLROnPlateau(patience=5,monitor='vall_loss',factor=0.1)

model_cnn.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),callbacks=[lr,early_stop])

In [ ]:
test=pd.read_csv('test.csv')

Predict

In [ ]:
test=test.values.reshape(-1,28,28,1)

In [ ]:
y_pred=model_cnn.predict(test)


In [ ]:
pred=np.argmax(y_pred,axis=-1)

In [ ]:
submit=pd.read_csv('/content/sample_submission.csv')

In [ ]:
submit['Label']=pred

In [ ]:
submit

In [ ]:
submit.to_csv('digit.csv', index=False)

In [ ]:
#same as submition above
y_pred = np.argmax(model_cnn.predict(test), axis=-1)

submission = pd.read_csv('/content/sample_submission.csv')
submission.Label = y_pred

submission.to_csv('submission.csv', index=False)